In [1]:
import re
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.sql.functions import col, lower, regexp_replace, split

def main():
    # Tạo Spark Session
    spark = SparkSession.builder \
        .appName("Spark Word2Vec Demo") \
        .master("local[*]") \
        .getOrCreate()

    print("Spark session initialized")

    #  Đọc dữ liệu JSON (C4 dataset)

    data_path = "/content/c4-train.00000-of-01024-30K.json"
    try:
        df = spark.read.json(data_path)
    except Exception as e:
        print(f"Lỗi đọc file: {e}")
        spark.stop()
        return

    print("Đã đọc dữ liệu JSON")
    print(f"Tổng số dòng: {df.count()}")

    # Tiền xử lý văn bản
    # Giữ cột 'text', đưa về lowercase, bỏ ký tự đặc biệt, rồi tách từ
    df_clean = df.select(lower(col("text")).alias("text"))
    df_clean = df_clean.withColumn(
        "text",
        regexp_replace(col("text"), r"[^a-zA-Z\s]", "")
    )
    df_clean = df_clean.withColumn(
        "words",
        split(col("text"), r"\s+")
    )

    print("Đã làm sạch & tách từ")

    # Cấu hình & huấn luyện Word2Vec
    word2vec = Word2Vec(
        vectorSize=100,
        minCount=5,
        inputCol="words",
        outputCol="word_vectors"
    )

    print("Đang huấn luyện mô hình Word2Vec...")
    model = word2vec.fit(df_clean)
    print("Huấn luyện xong")

    # Demo: tìm từ tương tự với “computer”
    print("\n Các từ tương tự với 'computer':")
    try:
        synonyms = model.findSynonyms("computer", 5)
        synonyms.show(truncate=False)
    except Exception as e:
        print("Từ 'computer' không tồn tại trong từ điển hoặc dữ liệu quá nhỏ.")
        print(e)

    # Dừng Spark
    spark.stop()
    print("Spark session stopped")

if __name__ == "__main__":
    main()


Spark session initialized
Đã đọc dữ liệu JSON
Tổng số dòng: 30000
Đã làm sạch & tách từ
Đang huấn luyện mô hình Word2Vec...
Huấn luyện xong

 Các từ tương tự với 'computer':
+---------+------------------+
|word     |similarity        |
+---------+------------------+
|uwowned  |0.7178223729133606|
|desktop  |0.7055313587188721|
|pc       |0.623419463634491 |
|computers|0.6192078590393066|
|software |0.6015800833702087|
+---------+------------------+

Spark session stopped
